In [1]:
from ap_utils import *

In [20]:
import numpy as np

In [21]:
from load_and_preprocess_data import load_data, get_label_appended_data

## Loading the data

In [22]:
path_to_data = '../data/test_data.txt'
min_support = 0.3
min_confidence = 0.3

In [23]:
data = load_data('../data/diabetes.csv')
order = [col for col in data.columns[:-1]]
transformed_data = get_label_appended_data(data)

In [24]:
transactions = transformed_data.to_numpy()[:100]

In [27]:
print(order)

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


## Initialization

In [28]:
C = {}
L = {}
itemset_size = 1
discarded = {itemset_size: []}
items = []
for transaction in transactions:
    for item in transaction[:-1]:
        items.append([item])
items.sort(key = lambda x: order.index(x[0].split(',')[1]))
C.update({itemset_size: items})

In [29]:
def print_table(T, supp_count):
    print('Itemset | Frequency')
    for k in range(len(T)):
        print(f'{T[k]} : {supp_count[k]}')
    print()
    print()

In [30]:
supp_count_L = {}
f, sup, new_discarded = get_frequent(C[itemset_size], transactions, min_support, discarded)
discarded.update({itemset_size: new_discarded})
L.update({itemset_size: f})
supp_count_L.update({itemset_size: sup})

In [31]:
print_table(L[1], supp_count_L[1])

Itemset | Frequency
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,SkinThickness'] : 32
['0,Insulin'] : 55
['0,Insulin'] : 55
['0,Insulin'] : 55
['0,Insulin'] : 55
['0,Insulin'] : 55
['0,Insulin'] : 55
['0,Insulin'] : 55
['0,Insulin'] : 55
['0,Insulin'] : 55
['0,Insul

In [32]:
k = itemset_size + 1
convergence = False
while convergence == False:
    C.update({k: join_set_itemsets(L[k - 1], order)})
    print(f'Table C{k}: \n')
    print(len(C[k]))
#     print_table(C[k], [count_occurences(it, transactions) for it in C[k]])
    f, sup, new_discarded = get_frequent(C[k], transactions, min_support, discarded)
    discarded.update({k: new_discarded})
    L.update({k: f})
    supp_count_L.update({k: sup})
    if len(L[k]) == 0:
        convergence = True
    else:
        print(f'Table L{k}: \n')
        print(len(L[k]))
#         print_table(L[k], supp_count_L[k])
    k += 1

Table C2: 

1760
Table L2: 

1760
Table C3: 

0


## Generate Rules

In [ ]:
from itertools import combinations, chain
def powerset(s):
    return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1)))

In [ ]:
def write_rules(X, X_S, S, conf, supp, lift, num_transactions):
    out_rules = ''
    out_rules += f'Freq. Itemset: {X}\n'
    out_rules += f'\tRule: {S} -> {X_S}\n'
    out_rules += f'\tConf: {0:2.3f} '.format(conf)
    out_rules += f'\tSupp: {0:2.3f} '.format(supp/num_trans)
    out_rules += f'\tLift: {0:2.3f}\n'.format(lift)
    return out_rules

In [ ]:
assoc_rules_str = ''
num_trans = len(transactions)
for i in range(1, len(L)):
    for j in range(len(L[i])):
        s = list(powerset(set(L[i][j])))
        s.pop()
        for z in s:
            S = set(z)
            X = set(L[i][j])
            X_S = set(X - S)
            sup_x = count_occurences(X, transactions)
            sup_x_s = count_occurences(X_S, transactions)
            conf = sup_x/count_occurences(S, transactions)
            lift = sup_x/(sup_x_s/num_trans)
            if conf >= min_confidence and sup_x >= min_support:
                assoc_rules_str += write_rules(X, X_S, S, conf, sup_x, lift, num_trans)

In [ ]:
print(assoc_rules_str)